# 0. 패키지 임포트

In [1]:
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from decouple import config
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

c:\Users\Admin\anaconda3\envs\llm-lab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. 기본 세팅

In [2]:
load_dotenv()

True

In [3]:
pc = Pinecone()

In [4]:
# embedding = OpenAIEmbeddings(model="text-embedding-3-large")
embedding = UpstageEmbeddings(model="solar-embedding-1-large")
index_name = config("PINECONE_INDEX")


In [5]:
database = PineconeVectorStore(
    embedding=embedding, 
    index_name=index_name
)

In [6]:
retriever = database.as_retriever(search_kwargs={"k": 3})
prompt = hub.pull('rlm/rag-prompt')
llm = ChatOpenAI(model="gpt-5-nano")

In [ ]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="법률의 이름. 예: '어선원 및 어선 재해보상보험법(법률)(제20132호)(20240724).docx'",
        type="string",
    ),
    AttributeInfo(
        name="chunk_type",
        description="텍스트 조각의 유형. 'title'(제목/총칙) 또는 'article'(개별 조항) 중 하나.",
        type="string",
    ),
]
document_content_description = "대한민국 법률 조항"

self_query_retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=database,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    verbose=True
)

In [32]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, 
    llm=llm,
)

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever, # 리트리버 방식 변경 가능 (retriever, retriever_from_llm, self_query_retriever )
    chain_type_kwargs={"prompt": prompt},
)

# 2. 선다형 문제로 검증 실행

In [19]:
df = pd.read_csv("객관식_문제.csv")
df.sample()

,Unnamed: 0,문제유형,난이도,문제내용,보기1,보기2,보기3,보기4,정답,해설
3126,자본시장과 금융투자업에 관한 법률 시행령(대통령령)(제35471호)(20250722),사지선다형,상,「자본시장과 금융투자업에 관한 법률 시행령(대통령령)(제35471호)(2025072...,원금손실 가능금액이 10% 초과,원금손실 가능금액이 20% 초과,원금손실 가능금액이 30% 초과,원금손실 가능금액이 50% 초과,2,「자본시장과 금융투자업에 관한 법률 시행령」 제2조제7항에서 '고난도금융투자상품'은...


In [20]:
success, fail = 0, 0
fail_questions = []
gpt = []
act = []
for i in range(100):
    sample = df.loc[i]
    question = """아래 문제를 보고 답을 고르시오.
만약 보기 중 3번째 보기가 정답이라면 3 이라고 출력하시오.
보기가 A,B,C,D 라면 1,2,3,4 라고 생각하고 답변하시오.
오직 숫자만 출력하시오.
출력 예시: 3
"""
    question += sample['문제내용'] + '\n'

    for i in range(1,5):
        col = '보기' + str(i)
        question += sample[col] + '\n'

    message = qa_chain.invoke({"query":question})['result']
    if message == str(sample['정답']):
        success += 1
    else:
        fail += 1
        fail_questions.append(sample['문제내용']) # 틀린문제만 추출해서 살펴보기
    gpt.append(message)
    act.append( str(sample['정답']))

OutputParserException: Parsing text
```json
{
    "query": "금융거래지표의 관리에 관한 법률 제10조에서 금지하는 행위는 무엇인가?",
    "filter": "eq(\\\"chunk_type\\\", \\\"article\\\")"
}
```
 raised following error:
No terminal matches '\' in the current parser context, at line 1 col 4

eq(\"chunk_type\", \"article\")
   ^
Expected one of: 
	* RPAR
	* DATE
	* SIGNED_INT
	* LSQB
	* CNAME
	* SIGNED_FLOAT
	* /'[^']*'/
	* ESCAPED_STRING
	* DATETIME

Previous tokens: Token('LPAR', '(')

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [21]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))
print('챗봇에 대답: ', gpt)
print('실제 답    : ', act)

정답 갯수 : 34
오답 갯수 : 40
정확도 : 0.4594594594594595
챗봇에 대답:  ['1', '모릅니다.', '모르겠습니다', '3', '모른다.', '2', '1', '모르겠습니다.', '모름', '3', '2', '모른다', '2', '3', '2', '2', '3', '1', '3', '3', '4', '2', '3', '3', '3', '4', '4', '모르겠습니다.', '1', '모르겠습니다.', '모릅니다.', '2', '4', '1', '4', '모른다.', '2', '1', '2', '모르겠습니다.', '모르겠습니다.', '1', '1', '1', '2', '2', '2', '4', '1', '1', '1', '3', '1', '1', '모른다.', '2', '1', '1', '정답을 알 수 없습니다.', '1', '2', '모르겠습니다.', '모르겠습니다', '모른다', '모른다.', '2', '모른다', '2', '모르겠습니다.', '모른다', '3', '3', '1', '모름']
실제 답    :  ['1', '2', '3', '3', '2', '2', '4', '3', '3', '3', '2', '1', '2', '3', '1', '2', '3', '1', '3', '2', '2', '3', '1', '3', '1', '1', '4', '2', '1', '1', '1', '2', '2', '1', '1', '3', '1', '1', '2', '1', '1', '1', '1', '1', '2', '2', '2', '1', '1', '1', '3', '3', '2', '1', '1', '1', '1', '1', '2', '2', '2', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '3', '1', '2']


## 2-1. 틀린문제 분석

In [64]:
df[[df['문제내용'] == q for q in fail_questions][0]]

,Unnamed: 0,문제유형,난이도,문제내용,보기1,보기2,보기3,보기4,정답,해설
10,개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령(대통령령)(제349...,사지선다형,중,「개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령(대통령령)(제34...,즉시 법원에 신고,지체 없이 시정 요구,채무자에게 통지,금융위원회에 보고,2,제25조에 따르면 지체 없이 시정을 요구해야 한다.


In [69]:
database.similarity_search_with_score(df[[df['문제내용'] == q for q in fail_questions][0]])

[(Document(id='a8ffed9b-ad0f-4d66-92f3-c92c335edc88', metadata={'chunk_type': 'article', 'source': '..\\..\\내파일\\은행법령\\한국은행 통화안정증권법(법률)(제14052호)(20160603).docx'}, page_content='제3조(무기명증권과 등록증권) ① 삭제 <2016. 3. 2.>\n\n②'),
  0.216716647),
 (Document(id='cda3dcb0-0597-42fb-a7e4-5c8af1193190', metadata={'chunk_type': 'article', 'source': '..\\..\\내파일\\상법투자자산증권주식법령\\한국은행 통화안정증권법(법률)(제14052호)(20160603).docx'}, page_content='제3조(무기명증권과 등록증권) ① 삭제 <2016. 3. 2.>\n\n②'),
  0.216452703),
 (Document(id='a41d6be0-998a-47d0-8a1b-c3be7561e7be', metadata={'chunk_type': 'article', 'source': '..\\..\\내파일\\금융법령\\자본시장과 금융투자업에 관한 법률(법률)(제20718호)(20250722).docx'}, page_content='제365조(명의개서대행회사의 등록) ① 증권의 명의개서를 대행하는 업무를 영위하려는 자는 금융위원회에 등록하여야 한다. <개정 2008. 2. 29.>\n\n② 제1항에 따른 등록을 하려는 자는 다음 각 호의 요건을 모두 갖추어야 한다.<개정 2016. 3. 22.>\n\n1. 전자등록기관 또는 전국적인 점포망을 갖춘 은행일 것\n\n2. 전산설비 등 대통령령으로 정하는 물적 설비를 갖출 것\n\n3. 대통령령으로 정하는 이해상충방지체계를 구축하고 있을 것\n\n③ 제1항에 따른 등록을 하려는 자는 금융위원회에 등록신청서를 제출하여야 한다.<개정 2008. 2. 29.>\n\n④ 금융위원회는 

# 3. 단답형 문제로 검증 실행

In [27]:
df = pd.read_csv("단답형_문제.csv")
df.sample()

,법령명,문제유형,난이도,문제내용,정답,해설
1091,외국인투자 촉진법 시행규칙(산업통상자원부령)(제00518호)(20230803),단답형,중,「외국인투자 촉진법 시행규칙(산업통상자원부령)(제00518호)(20230803)」 ...,별지 제20호 서식,제19조에 따르면 별지 제20호 서식의 보고서를 제출해야 한다.


In [28]:
results = []
SIMILARITY_THRESHOLD = 0.7

In [29]:
success, fail = 0, 0
gpt = []
act = []
for i in range(50):

    question = """아래 문제를 보고 짧고 간단하게 답을 단답식으로 제출하시오.
    좋은 답변들 예시 : ['대주주와의 거래', '지배구조법 대주주', '금융위원회', '제12조', '5%']
    이와같이 30자 이내로 핵심 단어의 단답형으로 답변하시오.
    """

    sample = df.loc[i]
    question += sample['문제내용']
    answer = sample['정답']


    
    message = qa_chain.invoke({"query": question})['result']

    ground_truth_embedding = embedding.embed_query(answer)
    prediction_embedding = embedding.embed_query(message)
    
    score = cosine_similarity(
        np.array(ground_truth_embedding).reshape(1, -1),
        np.array(prediction_embedding).reshape(1, -1)
    )[0][0]
    
    if score >= SIMILARITY_THRESHOLD:
        success += 1
    else:
        fail += 1
        
    gpt.append(message)
    act.append(answer)

In [30]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))
print('챗봇에 대답: ', gpt)
print('실제 답    : ', act)

정답 갯수 : 14
오답 갯수 : 36
정확도 : 0.28
챗봇에 대답:  ['3개월', '모르겠습니다.', '보호감시인', '모르겠습니다.', '보고 및 조치', '기초자료의수집및중요지표산정', '정관', '모른다.', '유의사항 공시, 대체지표 안내, 중단 안내', '주민등록자료', '은행', '국제금융기구에의 가입조치에 필요한 재원', '제2조제3항에 따른 경우', '공공기관 차입용 증권 발행', '출자금', '국회 의결', '임치소', '지급의무', '출자금 예산 반영 의무', '기획재정부장관', '국제금융기구 가입 및 협정이행', '개인금융채권을 보유한 자', '개인금융채무자보호제도정비', '이 법의 우선 적용', '금융거래를 하는 기관', '대규모 거래, 대체지표 부재, 타당성 저해 시', '중요지표 관리위원회', '모름', '국내행위및국내효과있는국외행위까지적용', '법', '정보 부족', '운영, 인식·평가·감시·통제, 분야별·소속별 위험한도·자본배분, 이해상충대책, 기타사항', '대통령령으로 정하는 사항', '요건 충족 시', '재무·경영상 위험 관리·감독', '응모총액을 발행총액으로 본다', '모집종료즉시납입', '현저한 통화팽창기', '액면발행, 할인발행, 할증발행', '발행총액 50%이상', '금융상품직접판매업, 모집인, 그 밖의 자', '자료의 제공 요청에 관한 사무', '금융상품을 올바르게 선택', '3가지 유형', '모름', '예금채권을 가진 자', '이사, 감사, 집행임원, 업무집행책임자', '자본확충을 위한 신속한 자금지원', '금융산업의 합리화', '6개월']
실제 답    :  ['3개월', '개인금융채무자의 권익과 금융질서', '업무의 분장 및 조직구조', '개인금융채무자', '지체 없이 시정 요구', '중요지표의 산출 방법 및 절차', '정관 또는 이에 준하는 규정', '전문지식과 경험', '산출업무규정의 내용', '기초자료 제출 방법 변경 사항', '새마을금고', '출자금 또는 출연금은 국제금융기구에 납입하는 금액이다.',